# Example: Gapminder dataset

https://www.gapminder.org/data/documentation/

Gapminder contains yearly data for countries worldwide. In this example we will be taking a look at the Life Expectancy per country throughout the years.

![Example gapminder trelliscope image](../_static/gapminder_example_output.png)

**Imports**

In [ ]:
import pandas as pd
import plotly.express as px

from trelliscope import Trelliscope
from trelliscope.facets import facet_panels
from trelliscope.state import NumberRangeFilterState
from trelliscope.examples.get_data import get_example_data

## Load dataset

In [ ]:
USE_SMALL_DATASET = True
gapminder = get_example_data("gapminder")

if USE_SMALL_DATASET:
    df = gapminder
else:
    df = gapminder
    
df.head()

## Create graphs

The gapminder data contains timeseries. Let's create a linegraph showing the Life Expectancy over time for each country.

### Generate facet plots

Grammar of Graphics. Use `facet_panels()` to create a Plotly graph for each small data frame. 

1. Decide how you need to group the data, in this case we group the data by `country`, `continent` and `iso_alpha2`. 
2. Use a plotting function to create a figure for each group. Here we use `px.line` from https://plotly.com/python/line-charts/

The output is a grouped dataframe, with a column containing `plotly.Figure` objects.

In [ ]:
df_panels = facet_panels(
    df=df,
    panel_column_name="lifeExp_time",
    facet_columns=["country", "continent", "iso_alpha2"],
    plot_function=px.line,
    params={"x": "year", "y": "lifeExp", "markers": True},
)
df_panels.head()

### Preview a single graph

We can select a single figure and preview it before we generate the entire Trelliscope display.

In [ ]:
# required to render plotly figure with nbconvert
from IPython.display import display, HTML
display(HTML('<script src="http://requirejs.org/docs/release/2.1.6/comments/require.js"></script>'))

In [ ]:
df_panels.loc[("United Kingdom", "Europe", "GB"), "lifeExp_time"].show()

## Create metadata

Each figure is joined with metadata about the data in the graph. To do this we group the data using the same groups as each figure, and then aggregate the data.

In [ ]:
# Aggregate statistics per panel
df_meta = (
    df
    .groupby(["country", "continent", "iso_alpha2"])
    .agg(
        mean_lifeExp=("lifeExp", "mean"),
        min_lifeExp=("lifeExp", "min"),
        max_lifeExp=("lifeExp", "max"),
        mean_gdp=("gdpPercap", "mean"),
        first_year=("year", "min"),
        latitude=("latitude", "first"),
        longitude=("longitude", "first"),
    )
    .reset_index()
)

# Add additional metadata to each group
df_meta = (
    df_meta
    .assign(
        first_data=pd.to_datetime(df_meta["first_year"], format="%Y"),
        wiki=df_meta["country"].apply(lambda x: f"https://en.wikipedia.org/wiki/{x}")
    )
)



## Join panels and metadata

Combine the panel figures and their data attributes into a single dataframe to create the `Trelliscope` input data.

In [ ]:
# Join metas with panels
df_joined = df_panels.merge(df_meta, on=["country", "continent", "iso_alpha2"], validate="1:1")
df_joined.head()

## Create Trelliscope

Set various parameters explicitly by piping method `Trelliscope`. In this example we create the `Trelliscope` and let it infer which column contains the figure, and which contain meta attributes.

Configure the initial settings when opening the Trelliscope display by setting the default label, layout, sorting and filter options. 

In [ ]:

tr = (
    Trelliscope(df_joined, name="gapminder", show_progress_bar=False)
    .set_default_labels(["country", "continent", "iso_alpha2"])
    .set_default_layout(3)
    .set_default_sort(
        ["continent", "mean_lifeExp"], sort_directions=["asc", "desc"]
    )
    .set_default_filters([NumberRangeFilterState("mean_lifeExp", 30, 60)])
    .write_display()
)

In [ ]:
# Open Trelliscope in a new tab
tr.view_trelliscope()

## Add secondary panel

We can add an additional image for each panel, such as the country flags.

### Add flags

We create a new dataframe with images in a column, referencing remote image files of country flags.

In [ ]:
# The flag column will hold references to the local files
flag_base_url = "https://raw.githubusercontent.com/hafen/countryflags/master/png/512/"
df_flags = (
    df_meta
    [["country", "continent", "iso_alpha2"]]
    .assign(
        # flag=df_meta["iso_alpha2"].apply(lambda x: (local_flags_path / f"{x}.png").as_posix()),
        flag=df_meta["iso_alpha2"].apply(lambda x: f"{flag_base_url}{x}.png")
    )
)
df_flags

## Update Trelliscope

In [ ]:
# Join metas with timeseries panels and flag panels
df_joined = (
    df_panels
    .merge(df_meta, on=["country", "continent", "iso_alpha2"], validate="1:1")
    .merge(df_flags, on=["country", "continent", "iso_alpha2"], validate="1:1")
)

# Write trelliscope
tr_with_flags = (
    Trelliscope(df_joined, name="gapminder", show_progress_bar=False)
    .set_default_labels(["country", "continent", "iso_alpha2"])
    .set_default_layout(3)
    .set_default_sort(
        ["continent", "mean_lifeExp"], sort_directions=["asc", "desc"]
    )
    .set_default_filters([NumberRangeFilterState("mean_lifeExp", 30, 60)])
    .write_display()
)

tr_with_flags.view_trelliscope()

#### Example output

You can highlight a panel and find the flag under `Show Additional Panels` in the top-left.

![Example gapminder with flags trelliscope image](../_static/gapminder_flag_example_output.png)